In [1]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_multigroup_params, load_params
from multi_group_simulation import MultiGroupSimulation

In [3]:
num_trajectories = 200

interaction_matrix = np.array([[12.5,4,0.1,0.1,1,0.05,0.05,0.1],
                               [3.41,8,0.1,0.1,1,0.05,0.05,0.2],
                               [0.19,0.22,4,0.1,1.2,0.05,0.2,1.8],
                               [0.14,0.17,0.07,9,1,0.05,0.05,0.2],
                               [1.92,2.26,1.22,1.37,1,0.15,0.3,1.56],
                               [0.18,0.21,0.1,0.13,0.28,1.8,0.2,1.56],
                               [0.07,0.09,0.15,0.05,0.23,0.08,1.8,1.56],
                               [0.011,0.026,0.106,0.016,0.091,0.048,0.12,3.5]])

interaction_matrix = 1.3 * interaction_matrix

group_names = ['UG (campus)', 'UG (off campus)', 'GS (research)', 'GS (other)', 'Faculty/Staff (student facing)', 'Faculty/Staff (non student facing)', 'Faculty/Staff (off campus)', 'Ithaca Community']


num_groups = len(group_names)
free_and_infectious_pess = np.zeros((num_trajectories, num_groups))
QS_pess = np.zeros((num_trajectories, num_groups))
QI_pess = np.zeros((num_trajectories, num_groups))
    
ug_dorm_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/ug_dorm.yaml')[1]
ug_off_campus_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/ug_off_campus.yaml')[1]
gs_research_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/grad_research.yaml')[1]
gs_other_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/grad_other.yaml')[1]
faculty_staff_student_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/faculty_staff_student_same_age.yaml')[1]
faculty_staff_non_student_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/faculty_staff_non_student_same_age.yaml')[1]
faculty_staff_off_campus_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/faculty_staff_off_campus_same_age.yaml')[1]
ithaca_community_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/pessimistic/ithaca_community.yaml')[1]

params_list = [ug_dorm_params.copy(), ug_off_campus_params.copy(), gs_research_params.copy(), gs_other_params.copy(), faculty_staff_student_params.copy(), faculty_staff_non_student_params.copy(), faculty_staff_off_campus_params.copy(), ithaca_community_params.copy()]

for index, params in enumerate(params_list):
    params['expected_contacts_per_day'] = interaction_matrix[index, index]

sim = MultiGroupSimulation(params_list, interaction_matrix, group_names)
    
for i in range(num_trajectories):
    sim.reset_initial_state()
    for _ in range(18):
        sim.step()

    for j in range(num_groups):
        free_and_infectious_pess[i, j] = sim.get_free_infectious(j)
        QS_pess[i, j] = sim.sims[j].QS
        QI_pess[i, j] = sim.sims[j].QI

print("Pessimistic: ")
print("free-and-infectious: ", np.mean(free_and_infectious_pess, axis=0))

populations = [sim.sims[j].pop_size for j in range(num_groups)]
print("prevalence:", np.mean(free_and_infectious_pess, axis=0) / populations)
print("QS: ", np.mean(QS_pess, axis=0))
print("QI: ", np.mean(QI_pess, axis=0))

Pessimistic: 
free-and-infectious:  [30.45  33.76   7.965  8.84  10.925  2.03   5.185 35.575]
prevalence: [0.00730742 0.00442058 0.0027513  0.00226203 0.00303641 0.0010645
 0.00108518 0.00057379]
QS:  [174.655 260.38   53.82   87.525  81.585  19.98   48.555 536.385]
QI:  [ 53.425  76.43   14.515  26.345  24.17    5.12    8.21  105.675]


In [4]:
# statistics within Cornell (including students, faculty / staff)
print(np.sum(np.mean(free_and_infectious_pess[:, :7], axis=0)))
print(np.sum(np.mean(QI_pess[:, :7], axis=0)))
print(np.sum(np.mean(QS_pess[:, :7], axis=0)))

99.155
208.215
726.5


In [6]:
# quarantine/isolation statistics of students
print(np.sum(np.mean(QI_pess[:, :4], axis=0)))
print(np.sum(np.mean(QS_pess[:, :4], axis=0)))

170.715
576.38


In [17]:
print(np.sum(np.percentile(QI_pess[:, :4], 90, axis=0)))
print(np.sum(np.percentile(QS_pess[:, :4], 90, axis=0)))

234.1
801.9


In [7]:
# quarantine/isolation statistics of on-campus students
print(np.sum(np.mean(QI_pess[:, 0], axis=0)))
print(np.sum(np.mean(QS_pess[:, 0], axis=0)))

53.425
174.655


In [19]:
print(np.sum(np.percentile(QI_pess[:, 0], 90, axis=0)))
print(np.sum(np.percentile(QS_pess[:, 0], 90, axis=0)))

78.0
261.0


In [8]:
# quarantine/isolation statistics of off-campus students
print(np.sum(np.mean(QI_pess[:, 1:4], axis=0)))
print(np.sum(np.mean(QS_pess[:, 1:4], axis=0)))


117.29
401.725


In [20]:
print(np.sum(np.percentile(QI_pess[:, 1:4], 90, axis=0)))
print(np.sum(np.percentile(QS_pess[:, 1:4], 90, axis=0)))

156.1
540.9


In [9]:
np.savetxt('/home/jw2529/results/free_and_infectious_pessimistic.out', free_and_infectious_pess)
np.savetxt('/home/jw2529/results/QI_pessimistic.out', QI_pess)
np.savetxt('/home/jw2529/results/QS_pessimistic.out', QS_pess)

In [10]:
num_trajectories = 200

interaction_matrix = np.array([[12.5,4,0.1,0.1,1,0.05,0.05,0.1],
                               [3.41,8,0.1,0.1,1,0.05,0.05,0.2],
                               [0.19,0.22,4,0.1,1.2,0.05,0.2,1.8],
                               [0.14,0.17,0.07,9,1,0.05,0.05,0.2],
                               [1.92,2.26,1.22,1.37,1,0.15,0.3,1.56],
                               [0.18,0.21,0.1,0.13,0.28,1.8,0.2,1.56],
                               [0.07,0.09,0.15,0.05,0.23,0.08,1.8,1.56],
                               [0.011,0.026,0.106,0.016,0.091,0.048,0.12,3.5]])

interaction_matrix = 0.7 * interaction_matrix

group_names = ['UG (campus)', 'UG (off campus)', 'GS (research)', 'GS (other)', 'Faculty/Staff (student facing)', 'Faculty/Staff (non student facing)', 'Faculty/Staff (off campus)', 'Ithaca Community']


num_groups = len(group_names)
free_and_infectious_opt = np.zeros((num_trajectories, num_groups))
QS_opt = np.zeros((num_trajectories, num_groups))
QI_opt = np.zeros((num_trajectories, num_groups))
    
ug_dorm_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/ug_dorm.yaml')[1]
ug_off_campus_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/ug_off_campus.yaml')[1]
gs_research_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/grad_research.yaml')[1]
gs_other_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/grad_other.yaml')[1]
faculty_staff_student_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/faculty_staff_student_same_age.yaml')[1]
faculty_staff_non_student_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/faculty_staff_non_student_same_age.yaml')[1]
faculty_staff_off_campus_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/faculty_staff_off_campus_same_age.yaml')[1]
ithaca_community_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/optimistic/ithaca_community.yaml')[1]

params_list = [ug_dorm_params.copy(), ug_off_campus_params.copy(), gs_research_params.copy(), gs_other_params.copy(), faculty_staff_student_params.copy(), faculty_staff_non_student_params.copy(), faculty_staff_off_campus_params.copy(), ithaca_community_params.copy()]

for index, params in enumerate(params_list):
    params['expected_contacts_per_day'] = interaction_matrix[index, index]

sim = MultiGroupSimulation(params_list, interaction_matrix, group_names)
    
for i in range(num_trajectories):
    sim.reset_initial_state()
    for _ in range(18):
        sim.step()

    for j in range(num_groups):
        free_and_infectious_opt[i, j] = sim.get_free_infectious(j)
        QS_opt[i, j] = sim.sims[j].QS
        QI_opt[i, j] = sim.sims[j].QI

print("Optimistic: ")
print("free-and-infectious: ", np.mean(free_and_infectious_opt, axis=0))

populations = [sim.sims[j].pop_size for j in range(num_groups)]
print("prevalence:", np.mean(free_and_infectious_opt, axis=0) / populations)
print("QS: ", np.mean(QS_opt, axis=0))
print("QI: ", np.mean(QI_opt, axis=0))

Optimistic: 
free-and-infectious:  [ 2.755  3.08   1.36   1.105  1.2    0.41   1.53  13.17 ]
prevalence: [0.00066115 0.0004033  0.00046978 0.00028275 0.00033352 0.000215
 0.00032022 0.00021242]
QS:  [14.31  21.84   4.785  8.745  5.625  2.625  7.62  80.295]
QI:  [15.16  20.91   5.415  9.77   7.28   2.605  3.89  42.04 ]


In [11]:
# statistics within Cornell (including students, faculty / staff)
print(np.sum(np.mean(free_and_infectious_opt[:, :7], axis=0)))
print(np.sum(np.mean(QI_opt[:, :7], axis=0)))
print(np.sum(np.mean(QS_opt[:, :7], axis=0)))

11.44
65.02999999999999
65.55


In [12]:
# quarantine/isolation statistics of students
print(np.sum(np.mean(QI_opt[:, :4], axis=0)))
print(np.sum(np.mean(QS_opt[:, :4], axis=0)))

51.254999999999995
49.68


In [21]:
# quarantine/isolation statistics of students
print(np.sum(np.percentile(QI_opt[:, :4], 90, axis=0)))
print(np.sum(np.percentile(QS_opt[:, :4], 90, axis=0)))

71.1
78.0


In [13]:
# quarantine/isolation statistics of on-campus students
print(np.sum(np.mean(QI_opt[:, 0], axis=0)))
print(np.sum(np.mean(QS_opt[:, 0], axis=0)))

15.16
14.31


In [22]:
# quarantine/isolation statistics of on-campus students
print(np.sum(np.percentile(QI_opt[:, 0], 90, axis=0)))
print(np.sum(np.percentile(QS_opt[:, 0], 90, axis=0)))

22.0
24.0


In [14]:
# quarantine/isolation statistics of off-campus students
print(np.sum(np.mean(QI_opt[:, 1:4], axis=0)))
print(np.sum(np.mean(QS_opt[:, 1:4], axis=0)))

36.095
35.37


In [23]:
# quarantine/isolation statistics of off-campus students
print(np.sum(np.percentile(QI_opt[:, 1:4], 90, axis=0)))
print(np.sum(np.percentile(QS_opt[:, 1:4], 90, axis=0)))

49.099999999999994
54.0


In [15]:
np.savetxt('/home/jw2529/results/free_and_infectious_optimistic.out', free_and_infectious_opt)
np.savetxt('/home/jw2529/results/QI_optimistic.out', QI_opt)
np.savetxt('/home/jw2529/results/QS_optimistic.out', QS_opt)